In [1]:
import numpy as np
import parsl
from parsl import python_app
from parsl.config import Config
from parsl.executors.threads import ThreadPoolExecutor
import pandas as pd
import os
import re
import pandas as pd
from transformers import BertModel, BertTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity
import time 
import threading
import hdbscan
import psutil
import GPUtil
from time import sleep
from bertopic import BERTopic
from datetime import datetime
from collections import Counter
from umap import UMAP

/home/lyncoln/Git/similaridade_workflow/venv_linux/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def monitor_resources(interval, path_name):
    while not stop_thread.is_set():
        # Obter o timestamp atual
        current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        # Monitorando CPU e memória
        cpu_usage = psutil.cpu_percent(interval=interval)
        memory_info = psutil.virtual_memory()
        memory_usage = memory_info.used / (1024 ** 2)  # Convertendo de bytes para MB

        # Monitorando GPU
        gpus = GPUtil.getGPUs()
        gpu_info = [(gpu.load, gpu.memoryUsed) for gpu in gpus]
        gpu_load = gpu_info[0][0]
        gpu_memory_used = gpu_info[0][1]

        new_row = pd.DataFrame([{
            'Timestamp': current_time,
            'CPU': cpu_usage,
            'Memory': memory_usage,
            'GPU': gpu_load,
            'GPU_Memory': gpu_memory_used
        }])

        if not os.path.isfile(path_name):
            df = pd.DataFrame(columns=["Timestamp", "CPU", "Memory", "GPU", "GPU_Memory"])
            df.to_csv(path_name, index=False)
        else:
            df = pd.read_csv(path_name)

        df = pd.concat([df, new_row], ignore_index=True)
        df.to_csv(path_name, index=False)

        # Limpando a memória
        del df

        sleep(interval)

In [3]:
torch.cuda.set_per_process_memory_fraction(0.99)
config = Config(executors=[ThreadPoolExecutor()])
parsl.load(config)

In [4]:
def extractName(text):
  return re.findall(r'"name"\s*:\s*"([^"]+)"',text)

def extractTag(text):
  matches = re.findall(r'\"tags\": \[(.*?)\]', text, re.DOTALL)
  matches = re.findall(r'\"(.*?)\"', matches[0].strip(','))
  return matches

def extractDescription(text):
  return re.findall(r'"description"\s*:\s*"([^"]+)"',text)

def extractLabel(text):
    return re.findall(r'"label"\s*:\s*"([^"]+)"', text)


In [5]:
os.chdir(os.path.join(os.pardir,os.pardir, 'workflows_galaxy'))
nomes_arquivos = []
diretorio = os.getcwd()
print(diretorio)
for item in os.listdir(diretorio):
    caminho_completo = os.path.join(diretorio, item)
    if os.path.isfile(caminho_completo):
        nomes_arquivos.append(item)

print(len(nomes_arquivos))

/home/lyncoln/Git/similaridade_workflow/workflows_galaxy
1014


In [6]:
dic_workflows = {}
@python_app
def processar_arquivo(arquivo):
    names = []
    tags = []
    descriptions = []
    labels = []

    # Carregar o arquivo JSON
    with open(arquivo, 'r') as file:
        fileName = file.name
        dados = file.read()

        # Extração das informações
        names.extend(extractName(dados))
        tags.extend(extractTag(dados))
        descriptions.extend(extractDescription(dados))
        labels.extend(extractLabel(dados))
        all_results = names + tags + descriptions + labels

        combined_results = {
        'Tags': tags if tags else [],  
        'text': all_results,
        }


    return combined_results

# Inicia o processamento dos arquivos em paralelo
futures = [processar_arquivo(arquivo) for arquivo in nomes_arquivos]

# Obter os resultados
resultados = [future.result() for future in futures]

for arquivo, all_results in zip(nomes_arquivos, resultados):
  dic_workflows[arquivo] = all_results

In [7]:
df = pd.DataFrame.from_dict(dic_workflows, orient='index').reset_index()
df

index                    Tags  \
0     0c86c39dcd9e08c6.json                      []   
1     692a2b0bb818336d.json                      []   
2     3680984663c813e1.json                      []   
3     49f8b32c3206f76c.json  [variant, snps, human]   
4     25adc55d2a26e34b.json                      []   
...                     ...                     ...   
1009  d5cba8a5ba6880fd.json                      []   
1010  e0da87cadb1e6d5f.json                      []   
1011  fbf75fbb72b488bd.json                      []   
1012  0d8eb75d28fa7df2.json                      []   
1013  f8238234db6f04c3.json                      []   

                                                   text  
0                                          [Project_CP]  
1     [Workflow constructed from history 'K22063917 ...  
2     [dhfr, Bowtie2, output, bowtie2, MPileup, outp...  
3     [Workflow for Genomic Data Science with Galaxy...  
4     [GigaScience Example 1B -- aye-aye FST (import...  
...                                                 ...  
1009  [BRACA2 - primer design, exon, Input dataset, ...  
1010  [2 peaks-result, data file 2 and more peaks 16...  
1011  ['BBL735_Lab2(Olympic)_AT', olympics.tsv, Inpu...  
1012  [Desanka Lazic_Project, Coriell-NA12880_R2.fas...  
1013  [handson, Input dataset, Input dataset, Join, ...  

[1014 rows x 3 columns]

In [8]:
os.chdir("..")
os.chdir("scr")
os.chdir("similaridade")

top_x_list = list(range(9,11))  

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:

df = pd.DataFrame.from_dict(dic_workflows, orient='index').reset_index()
for top_x in top_x_list:

    # Medir o tempo de execução
    start_time = time.time()

    stop_thread = threading.Event()
    monitor_thread = threading.Thread(target=monitor_resources, args=(1.0,f"topicos/bert/hardware_top{top_x}_topico_descricao.csv",))    
    monitor_thread.daemon = True
    monitor_thread.start()

    # Carregar o modelo e o tokenizer

    df[f'top{top_x}_topic_bert'] = ""

    tokenizer = BertTokenizer.from_pretrained('google-bert/bert-large-uncased')
    model = BertModel.from_pretrained('google-bert/bert-large-uncased')

    # Certifique-se de que todas as colunas necessárias existam
    df[f'top{top_x}_topic'] = df.get(f'top{top_x}_topic', None)
    df[f'top{top_x}_topic_bert'] = df.get(f'top{top_x}_topic_bert', None)

    for i in range(len(df['text'])):
        print(i)
        # Contar o número de palavras no texto atual
        list_text = df['text'][i]
        n_words = sum(len(words.split()) for words in list_text)

        if n_words > 10:
            # Instanciar o BERTopic
            topic_model = BERTopic(embedding_model=model, nr_topics=2, hdbscan_model=hdbscan.HDBSCAN(min_cluster_size=2), umap_model=UMAP(n_components=2))
            topics_total, probabilities = topic_model.fit_transform(df['text'][i])

            topics = topic_model.get_topics()

            if topics:
                if len(topic_model.get_topics()) > 2:
                    topic_model.reduce_topics(df['text'][i], nr_topics=2)
                    topics = topic_model.get_topics()

                # Supondo que você quer as palavras do tópico mais comum, que geralmente é '0' ou o tópico com maior número de documentos
                top_topic_id = max(topics, key=lambda x: len(topics[x]))  # Este código pega o ID do tópico com mais palavras
                top_topic_words = topics[top_topic_id]  # Acessar as palavras do tópico mais comum
                top_words = [word for word, _ in top_topic_words[:10]]  # Pegar as 10 palavras mais significativas
                df[f'top{top_x}_topic_bert'][i] = top_words
                df[f'top{top_x}_topic'][i] = top_topic_id

            else:
                # Se não há tópicos, extraia as 10 palavras mais frequentes
                word_counts = Counter(list_text.split())
                most_common_words = [word for word, count in word_counts.most_common(10)]
                df[f'top{top_x}_topic_bert'][i] = most_common_words

        else: #Se tem menos de 10 palavras, então o texto é o próprio tópico
            df[f'top{top_x}_topic_bert'][i] = df['text'][i]

        topic_model = ''

    # Definir o dispositivo (GPU ou CPU)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    def text_to_embedding(text):
        encoded_input = tokenizer(text, return_tensors='pt', max_length=512, padding=True, truncation=True).to(device)
        with torch.no_grad():
            model_output = model(**encoded_input)
        # Pegar a média dos embeddings de todos os tokens para representar o texto
        return model_output.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

    df[f'top{top_x}_topic_bert_str'] = df[f'top{top_x}_topic_bert'].apply(lambda x: ' '.join(x))
    embeddings = np.vstack(df[f'top{top_x}_topic_bert_str'].apply(text_to_embedding))

    # Calcular a similaridade de cosseno entre todos os vetores
    cosine_sim = cosine_similarity(embeddings)

    embeddings_array = np.vstack(embeddings)

    # Calcular a matriz de similaridade
    similarity_matrix = cosine_similarity(embeddings_array)

    # Identificar os índices dos textos mais similares para cada texto
    similar_indices = similarity_matrix.argsort(axis=1)[:, :-top_x-2:-1]  # Selecionar os top_x mais similares excluindo o próprio texto

    # Remover o índice do próprio texto
    corrected_similar_indices = []
    corrected_similar_tags = []
    for idx, indices in enumerate(similar_indices):
        filtered_indices = [index for index in indices if index != idx][:top_x]  # Exclui o próprio e pega os top_x mais similares
        filtered_tags = [df.iloc[index]['Tags'] for index in filtered_indices]  # Obter as tags dos textos mais similares
        corrected_similar_indices.append(filtered_indices)
        corrected_similar_tags.append(filtered_tags)

    # Criar coluna no DataFrame para os índices dos textos mais similares
    df[f'top{top_x}_descricao'] = corrected_similar_indices

    # Criar coluna no DataFrame para as tags dos textos mais similares
    df[f'top{top_x}_tags_descricao'] = corrected_similar_tags

    # Calcular a média das similaridades dos textos mais similares para cada texto
    mean_similarities = []
    for idx, indices in enumerate(corrected_similar_indices):
        similarities = [similarity_matrix[idx, i] for i in indices]
        mean_similarity = np.mean(similarities)
        mean_similarities.append(mean_similarity)

    df[f'mean_similarity_top{top_x}_descricao'] = mean_similarities

    torch.cuda.empty_cache()
    stop_thread.set()
    monitor_thread.join()

    df.to_csv(f"topicos/bert/results_topicos_top{top_x}bert.csv")

In [9]:

top_x_list = list(range(3,11))  

df = pd.DataFrame.from_dict(dic_workflows, orient='index').reset_index()
for top_x in top_x_list:

    # Medir o tempo de execução
    start_time = time.time()

    stop_thread = threading.Event()
    monitor_thread = threading.Thread(target=monitor_resources, args=(1.0,f"topicos/scibert/hardware_top{top_x}_topico_descricao_scibert.csv",))    
    monitor_thread.daemon = True
    monitor_thread.start()

    # Carregar o modelo e o tokenizer

    df[f'top{top_x}_topic_scibert'] = ""

    tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
    model = BertModel.from_pretrained('allenai/scibert_scivocab_uncased')

    # Certifique-se de que todas as colunas necessárias existam
    df[f'top{top_x}_topic'] = df.get(f'top{top_x}_topic', None)
    df[f'top{top_x}_topic_scibert'] = df.get(f'top{top_x}_topic_scibert', None)

    for i in range(len(df['text'])):
        print(i)
        # Contar o número de palavras no texto atual
        list_text = df['text'][i]
        n_words = sum(len(words.split()) for words in list_text)

        if n_words > 10:
            # Instanciar o BERTopic
            topic_model = BERTopic(embedding_model=model, nr_topics=2, hdbscan_model=hdbscan.HDBSCAN(min_cluster_size=2), umap_model=UMAP(n_components=2))
            topics_total, probabilities = topic_model.fit_transform(df['text'][i])

            topics = topic_model.get_topics()

            if topics:
                if len(topic_model.get_topics()) > 2:
                    topic_model.reduce_topics(df['text'][i], nr_topics=2)
                    topics = topic_model.get_topics()

                # Supondo que você quer as palavras do tópico mais comum, que geralmente é '0' ou o tópico com maior número de documentos
                top_topic_id = max(topics, key=lambda x: len(topics[x]))  # Este código pega o ID do tópico com mais palavras
                top_topic_words = topics[top_topic_id]  # Acessar as palavras do tópico mais comum
                top_words = [word for word, _ in top_topic_words[:10]]  # Pegar as 10 palavras mais significativas
                df[f'top{top_x}_topic_scibert'][i] = top_words
                df[f'top{top_x}_topic'][i] = top_topic_id

            else:
                # Se não há tópicos, extraia as 10 palavras mais frequentes
                word_counts = Counter(list_text.split())
                most_common_words = [word for word, count in word_counts.most_common(10)]
                df[f'top{top_x}_topic_scibert'][i] = most_common_words

        else: #Se tem menos de 10 palavras, então o texto é o próprio tópico
            df[f'top{top_x}_topic_scibert'][i] = df['text'][i]

        topic_model = ''

    # Definir o dispositivo (GPU ou CPU)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    def text_to_embedding(text):
        encoded_input = tokenizer(text, return_tensors='pt', max_length=512, padding=True, truncation=True).to(device)
        with torch.no_grad():
            model_output = model(**encoded_input)
        # Pegar a média dos embeddings de todos os tokens para representar o texto
        return model_output.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

    df[f'top{top_x}_topic_scibert_str'] = df[f'top{top_x}_topic_scibert'].apply(lambda x: ' '.join(x))
    embeddings = np.vstack(df[f'top{top_x}_topic_scibert_str'].apply(text_to_embedding))

    # Calcular a similaridade de cosseno entre todos os vetores
    cosine_sim = cosine_similarity(embeddings)

    embeddings_array = np.vstack(embeddings)

    # Calcular a matriz de similaridade
    similarity_matrix = cosine_similarity(embeddings_array)

    # Identificar os índices dos textos mais similares para cada texto
    similar_indices = similarity_matrix.argsort(axis=1)[:, :-top_x-2:-1]  # Selecionar os top_x mais similares excluindo o próprio texto

    # Remover o índice do próprio texto
    corrected_similar_indices = []
    corrected_similar_tags = []
    for idx, indices in enumerate(similar_indices):
        filtered_indices = [index for index in indices if index != idx][:top_x]  # Exclui o próprio e pega os top_x mais similares
        filtered_tags = [df.iloc[index]['Tags'] for index in filtered_indices]  # Obter as tags dos textos mais similares
        corrected_similar_indices.append(filtered_indices)
        corrected_similar_tags.append(filtered_tags)

    # Criar coluna no DataFrame para os índices dos textos mais similares
    df[f'top{top_x}_descricao'] = corrected_similar_indices

    # Criar coluna no DataFrame para as tags dos textos mais similares
    df[f'top{top_x}_tags_descricao'] = corrected_similar_tags

    # Calcular a média das similaridades dos textos mais similares para cada texto
    mean_similarities = []
    for idx, indices in enumerate(corrected_similar_indices):
        similarities = [similarity_matrix[idx, i] for i in indices]
        mean_similarity = np.mean(similarities)
        mean_similarities.append(mean_similarity)

    df[f'mean_similarity_top{top_x}_descricao'] = mean_similarities

    torch.cuda.empty_cache()
    stop_thread.set()
    monitor_thread.join()

    df.to_csv(f"topicos/scibert/results_topicos_top{top_x}scibert.csv")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

: 